In [1]:
#Import Relevant Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
%matplotlib inline
from sklearn.model_selection import train_test_split
import keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, Input, LSTM, AveragePooling2D
from keras.datasets import mnist
from keras import regularizers, initializers, optimizers
import os
import datetime
from keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
import keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, Input, LSTM
from keras.datasets import mnist
from keras import regularizers, initializers, optimizers
import os
import datetime
from keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.callbacks import EarlyStopping
from pathlib import Path



from pathlib import Path

from sklearn.metrics import roc_auc_score, plot_confusion_matrix
import seaborn as sns

import gc
from timeit import default_timer as timer
import tensorflow as tf
pd.set_option('display.float_format', lambda x: '%.1f' % x)
os.environ['KMP_DUPLICATE_LIB_OK']='True' #This prevents kernel shut down due to xgboost conflict
from tensorflow.keras.preprocessing import image


import zipfile
import random

import cv2
from tensorflow import keras



from tensorflow.keras.layers import Conv2D, Dense, Flatten, Dropout, Activation
from tensorflow.keras.layers import BatchNormalization, Reshape, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


print(os.listdir("../input/aerial-cactus-identification/"))

In [2]:


    


def visualize_training_results(history):
    '''
    From https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/
    
    Input: keras history object (output from trained model)
    '''
    fig, (ax1, ax2) = plt.subplots(2, sharex=True)
    fig.suptitle('Model Results')

    # summarize history for accuracy
    ax1.plot(history.history['accuracy'])
    ax1.plot(history.history['val_accuracy'])
    ax1.set_ylabel('Accuracy')
    ax1.legend(['train', 'test'], loc='upper left')
    # summarize history for loss
    ax2.plot(history.history['loss'])
    ax2.plot(history.history['val_loss'])
    ax2.set_ylabel('Loss')
    ax2.legend(['train', 'test'], loc='upper left')
    
    plt.xlabel('Epoch')
    plt.show()
    
def plot_performance(hist):
    """ Returns 4 plots comparing Training and Validation data. 
    First plot returns training and validation accuracy. 
    Second plot returns training and validation loss. 
    Third plot returns training and validation F1-Scores. 
    Fourth plot returns training and validation recall scores. 
    
    hist: input history model containing train images, labels, and validation data. t"""
    
    hist_ = hist.history
    epochs = hist.epoch
    
    plt.plot(epochs, hist_['accuracy'], label='Training Accuracy')
    plt.plot(epochs, hist_['val_accuracy'], label='Validation Accuracy')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.figure()
    
    plt.plot(epochs, hist_['loss'], label='Training loss')
    plt.plot(epochs, hist_['val_loss'], label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    recall = np.array(hist_['recall'])
    precision = np.array(hist_['precision'])
    val_recall = np.array(hist_['val_recall'])
    val_precision = np.array(hist_['val_precision'])
    plt.figure()
    
    plt.plot(epochs, 
             2*((recall * precision)/(recall + precision)), 
             label='Training f1')
    plt.plot(epochs, 
             2*((val_recall * val_precision)/(val_recall + val_precision)), 
             label='Validation f1')
    plt.title('Training and validation F1-Score')
    plt.legend()
    plt.figure()
    
    plt.plot(epochs, recall, label = "Training Recall")
    plt.plot(epochs, val_recall, label = "Validation Recall")
    plt.title("Training and Validation Recall Scores")
    plt.legend()
    plt.figure()
    
    plt.show()
    


In [3]:
# Extract images from zip files
# (Not sure why I have to do this as everywhere seems to say that zip archives are automatically extracted... but oh well)

# Need to extract them to /kaggle/temp/ directory (instead of /kaggle/working/)
# Otherwise won't let me submit submission.csv file, instead generates output visulations of the images

with zipfile.ZipFile("../input/aerial-cactus-identification/train.zip","r") as z:
    z.extractall("/kaggle/temp/")
with zipfile.ZipFile("../input/aerial-cactus-identification/test.zip","r") as z:
    z.extractall("/kaggle/temp/test/") # needs to be in subdirectory (i.e. test/test/) for flow_from_directory to work

# for dirname, _, filenames in os.walk("./train"):
#     for filename in filenames[:5]:
#         print(os.path.join(dirname, filename))

print(len(os.listdir("../temp/train")))
print(len(os.listdir("../temp/test/test")))

In [4]:
train_dir = "../temp/train"
test_dir = "../temp/test"
labels = pd.read_csv('../input/aerial-cactus-identification/train.csv')

labels.has_cactus = labels.has_cactus.astype(str) # Classes must be str and not int
print(labels['has_cactus'].value_counts())

In [5]:

# Plot random sample of training images

rand_images = random.sample(os.listdir(train_dir), 16)

fig = plt.figure(figsize=(16,4))
for i, im in enumerate(rand_images):
    plt.subplot(2, 8, i+1)
    im = cv2.imread(os.path.join(train_dir, im))
    plt.imshow(im)
    plt.axis('off')
plt.show()

In [6]:
validation_split = 0.8
idxs = np.random.permutation(range(len(labels))) < validation_split*len(labels)

train_labels = labels[idxs]
val_labels = labels[~idxs]
print(len(train_labels), len(val_labels))

In [7]:
# Process image JPEGs into tensors
# Pixel values rescaled from [0,255] to [0,1]

# Generate batches of tensor image data (with real-time data augmentation - horizontal and vertical flips)
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255, horizontal_flip=True, vertical_flip=True)

batch_size = 128

train_generator = train_datagen.flow_from_dataframe(train_labels,directory=train_dir,x_col='id',
                                                    y_col='has_cactus',class_mode='binary',batch_size=batch_size,
                                                    target_size=(32,32))
val_generator = train_datagen.flow_from_dataframe(val_labels,directory=train_dir,x_col='id',
                                                    y_col='has_cactus',class_mode='binary',batch_size=batch_size,
                                                    target_size=(32,32))

In [8]:
input_shape = (32, 32, 3)

In [9]:
#First simple model
model_start = Sequential(
    [
        Input(input_shape),
        Flatten(), # need to flatten our images to be one long array
        Dense(64,activation="relu"),
        Dense(1, activation="sigmoid"),  
        
    ])

model_start.summary()

In [10]:
model_start.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy', 'Recall', 'Precision'])

In [11]:
history = model_start.fit(train_generator, 
                     batch_size=batch_size,
                     epochs=5, 
                     validation_data=val_generator)

In [12]:
score1 = model_start.evaluate(val_generator, verbose=0)
print("Test loss:", score1[0])
print("Test accuracy:", score1[1])
print("Test recall:", score1[2])
print("Test precision:", score1[3])
visualize_training_results(history)
plot_performance(history)


In [ ]:
model_start = Sequential(
    [
        Input(input_shape),
        Flatten(), # need to flatten our images to be one long array
        Dense(64,activation="relu"),
        Dense(1, activation="sigmoid"),  
        
    ])

model_start.summary()

In [25]:
model2 = Sequential([

        Conv2D(32, (3,3), activation = 'relu', padding = 'same', input_shape = input_shape),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation = 'relu'),
        Dense(1, activation='sigmoid')

]
)
opt = keras.optimizers.Adam(learning_rate=0.01)

# compiling models
model2.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy', 'Recall', 'Precision'])

# early stopping
cp = EarlyStopping(patience = 5, restore_best_weights=True)

model2.summary()

In [26]:
start = timer()

history2 = model2.fit(train_generator,
                    batch_size = batch_size,
                    epochs=50, 
                    validation_data = val_generator,
                    callbacks = [cp]
                    
                  
                   )
end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

In [28]:
score2 = model2.evaluate(val_generator, verbose=0)
print("Test loss:", score2[0])
print("Test accuracy:", score2[1])
print("Test recall:", score2[2])
print("Test precision:", score2[3])
visualize_training_results(history2)
plot_performance(history2)

In [29]:
model3 = Sequential([

        Conv2D(32, (3,3), activation = 'relu', padding = 'same', input_shape = input_shape),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation = 'relu'),
        Dense(1, activation='sigmoid')

]
)

model3.summary()

In [31]:
model3.compile(loss = keras.losses.binary_crossentropy,
              optimizer = 'adam',
              metrics = ['accuracy', 'Recall', 'Precision'])
#Very cool early stopping and learning rate work
cp = [EarlyStopping(monitor='val_loss', patience=20, verbose=1, restore_best_weights=True), # Stop training when a monitored metric has stopped improving
             ReduceLROnPlateau(patience=10, verbose=1), # Reduce learning rate when a metric has stopped improving

            ]

In [32]:
history3 = model3.fit(train_generator,
                    epochs = 100,
                    verbose = 1,
                    callbacks = cp,
                    validation_data = val_generator,
                    
                   )

In [33]:
score3 = model3.evaluate(val_generator, verbose=0)
print("Test loss:", score3[0])
print("Test accuracy:", score3[1])
print("Test recall:", score3[2])
print("Test precision:", score3[3])
visualize_training_results(history3)
plot_performance(history3)

In [34]:


model6 = Sequential([

    Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), padding='same', activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), padding='same', activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), padding='same', activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
    
])

model6.summary()

In [36]:
model6.compile(loss = keras.losses.binary_crossentropy,
              optimizer = 'adam',
              metrics = ['accuracy', 'Recall', 'Precision'])

cp = [EarlyStopping(monitor='val_loss', patience=20, verbose=1, restore_best_weights=True), # Stop training when a monitored metric has stopped improving
             ReduceLROnPlateau(patience=10, verbose=1), # Reduce learning rate when a metric has stopped improving

            ]

In [37]:
history6 = model6.fit(train_generator,
                    epochs = 100,
                    verbose = 1,
                    callbacks = cp,
                    validation_data = val_generator,
                    #class_weight = class_weights,
                   )

In [38]:
score6 = model6.evaluate(val_generator, verbose=0)
print("Test loss:", score6[0])
print("Test accuracy:", score6[1])
print("Test recall:", score6[2])
print("Test precision:", score6[3])
visualize_training_results(history6)
plot_performance(history6)